In [ ]:
#| default_exp photomapper

# PhotoMapper

> API details.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import base64
from os.path import exists
from pathlib import Path

from exif import Image
import folium
from folium import IFrame

import matplotlib
import numpy as np
import pandas as pd
import PIL

In [ ]:
#| export
def coords_to_decimal(coords: tuple[float,float],  # (45.5454545, -115.12312312)
                      ref: str  # "N", "S", "E", "W"
                     ) -> float:
    """Covert a tuple of coordinates in the format (degrees, minutes, seconds)
    and a reference to a decimal representation.
    
    Returns:
        float: A signed float of decimal representation of the coordinate.
    """
    if ref.upper() in ['W', 'S']:
        mul = -1
    elif ref.upper() in ['E', 'N']:
        mul = 1
    else:
        print(f"Expecting one of 'N', 'S', 'E' or 'W', got {ref} instead.")
        
    return mul * (coords[0] + coords[1] / 60 + coords[2] / 3600) 


def get_decimal_coord_from_exif(exif_data: Image  # takes an Exif Image
                               ) -> tuple[float, float, float]:
    """Get coordinate data from exif and convert to a tuple of 
    decimal latitude, longitude and altitude.
    Returns:
        tuple[float, float, float]: A tuple of decimal coordinates (lat, lon, alt)
    """
    try:
        lat = coords_to_decimal(
            exif_data['gps_latitude'], 
            exif_data['gps_latitude_ref']
            )
        lon = coords_to_decimal(
            exif_data['gps_longitude'], 
            exif_data['gps_longitude_ref']
            )
        alt = exif_data['gps_altitude']
        return (lat, lon, alt)
    except (AttributeError, KeyError):
        print('Image does not contain gps data or data is invalid.')  
        raise
        
        
def get_caption_from_exif(exif_data: Image  # takes an Exif Image
                               ) -> str:
    """Get caption data from exif and convert to a string.
    Returns:
        str: caption
    """
    try:
        return exif_data['caption'] 
    except (AttributeError, KeyError):
        return ""


def get_gps_data_from_images(image_path: Path,  # "Path object"
                             image_extension: str = '*.jpg'  # image extensions glob
                            ) -> dict[str, dict]:
    """Create a dictionary of gps data from images in a directory.
    Returns:
        dict[str, dict]: A dictionary with filename as the key
            and a value of a dictionary if the format:
                {
                    'coordinates': tuple[float, ...],
                    'timestamp': str
                }
    """
    coord_dict = dict()
    source_files = [f for f in image_path.rglob(image_extension)]
    exif = [Image(open(f, 'rb')) for f in source_files]
    
    for f, data in zip(source_files, exif):
        try:
            coord = get_decimal_coord_from_exif(data)
            caption = get_caption_from_exif(data)
        except (AttributeError, KeyError):
            continue
        else:
            coord_dict[str(f)] = dict()
            coord_dict[str(f)]['img'] = f
            coord_dict[str(f)]['caption'] = caption
            coord_dict[str(f)]['latitude'] = coord[0]
            coord_dict[str(f)]['longitude'] = coord[1]
            coord_dict[str(f)]['altitude'] = coord[2]
        # Also read date when photo was taken (if available)
        try:
            coord_dict[str(f)]['timestamp'] = data.datetime
        except (AttributeError, KeyError):
            print(f"Photo {f.name} does not contain datetime information.")
            coord_dict[str(f)]['timestamp'] = None
    
    return coord_dict


def make_image_thumb(image: Image, thumbnail_path: Path) -> str:
    """ makes thumbnails if they aren't created already or returns the path """
    thumbnail_path = Path(f"{thumbnail_path}/{image.name}")
    if exists(thumbnail_path):
        return thumbnail_path  # don't bother reprocessing files
    MAX_SIZE = (300, 300)  # thumbnail size
    thumb = PIL.Image.open(image)
    thumb.thumbnail(MAX_SIZE)
    thumb.save(thumbnail_path)
    return thumbnail_path


def process_thumbnails(resources: dict, thumbnail_path: Path)-> dict[str, dict]:
    """ Processes the thumbnails, and updates the resource dict with thumb location"""
    for k, v in resources.items():
        v['thumb'] = make_image_thumb(v['img'], thumbnail_path)
    return resources


def make_dataframe(resources: dict) -> pd.DataFrame:
    # create dataframe from extracted data
    df = pd.DataFrame(resources).T
    df['timestamp'] = pd.to_datetime(df.timestamp, format="%Y:%m:%d %H:%M:%S")
    df.sort_values('timestamp', inplace=True)

    df['color_mapping'] = np.linspace(0, 1, num=df.shape[0])
    return df

In [ ]:
image_path = Path("output")
resources = get_gps_data_from_images(image_path)
df = make_dataframe(resources)
df
assert len(df) 1

Photo MVIMG_20181126_132952-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_132952-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS-edited.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS-edited.jpg does not contain datetime information.


124

Colors need to be converted to hex to work with folium

In [ ]:
#| export
def rgba_to_hex(rgba: tuple[float, ...]):
    return ('#{:02X}{:02X}{:02X}').format(*rgba[:3])

maybe nab some text from exif to be able to populate some text in here

In [ ]:
#| export
def make_html(image: Path, thumbnail: Path, caption: str) -> str:
    encoded_thumb = base64.b64encode(open(thumbnail, 'rb').read())
    html = f'<p> {caption}: <a href="http://localhost:8888/view/{image}" target="_blank">test link</a> <img src="data:image/jpeg;base64,{encoded_thumb.decode("UTF-8")}">'
    return html


def generate_map(df: pd.DataFrame) -> folium.Map:
    m = folium.Map()
    
    # get colour map
    cmap = matplotlib.cm.get_cmap('plasma')
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)

    # Add markers
    resolution, width, height = 75, 7, 3
    for img, thumb, caption, lat, lon, col, date in zip(df.img,
                                               df.thumb,
                                               df.caption,
                                               df.latitude.values, 
                                               df.longitude.values, 
                                               df.color_mapping.values, 
                                               df.timestamp.values):

        iframe = IFrame(make_html(img, thumb, caption), width=(width*resolution)+20, height=(height*resolution)+20)
        popup = folium.Popup(iframe, max_width=2650)

        folium.CircleMarker(
            [lat, lon], 
            color=rgba_to_hex(cmap(col, bytes=True)), 
            fill_color=rgba_to_hex(cmap(col, bytes=True)),
            radius=6,
            popup = folium.Popup(iframe, max_width=2650),
            tooltip = np.datetime_as_string(date, unit='m')
            ).add_to(m)
    # create bounding box SW corner and NE corner, add 3 for padding
    sw = (df.latitude.max() + 3, df.longitude.min() - 3)
    ne = (df.latitude.min() - 3, df.longitude.max() + 3)
    m.fit_bounds([sw, ne])
    return m

Stitching all the pieces together, we take as inputs a path to images, and a path to our thumbnails folder, and we get a map return

In [ ]:
#| export
def process_images_into_map(image_path: Path, thumbnail_path: Path) -> folium.Map:
    resources = get_gps_data_from_images(image_path)
    # update the resources dict with thumbnail values / generate thumbnails
    resources = process_thumbnails(resources, thumbnail_path)
    df = make_dataframe(resources)
    m = generate_map(df)
    # m._repr_html_()  # dumps out the html
    return m

In [ ]:
image_path = Path("output")
thumbnail_path = Path("thumbnails")
m = process_images_into_map(image_path, thumbnail_path)

Photo MVIMG_20181126_132952-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_132952-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS-edited.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS-edited.jpg does not contain datetime information.


In [ ]:
m

In [ ]:
from nbdev import nbdev_export
nbdev_export()